In [16]:
from kubernetes import client, config
from kubernetes.client.rest import ApiException
import os

def find_node_ip_by_label(label_selector="APP=MONITORING", preferred_ip_type="InternalIP"):
    """
    지정된 레이블 셀렉터와 일치하는 첫 번째 노드의 IP 주소를 찾습니다.

    Args:
        label_selector (str): 노드를 필터링할 레이블 셀렉터 (예: 'APP=MONITORING').
        preferred_ip_type (str): 선호하는 IP 주소 유형 ('InternalIP' 또는 'ExternalIP').

    Returns:
        str: 찾은 노드의 IP 주소. 없으면 None을 반환합니다.
    """
    try:
        # 1. 쿠버네티스 설정 로드 시도
        #    - 클러스터 내부에서 실행 시도
        #    - 실패 시 로컬 kubeconfig 파일 사용 시도
        try:
            config.load_incluster_config()
            print("정보: 클러스터 내부 설정을 로드했습니다.")
        except config.ConfigException:
            try:
                config.load_kube_config()
                print("정보: Kubeconfig 파일을 로드했습니다.")
            except config.ConfigException:
                print("오류: 쿠버네티스 설정을 로드할 수 없습니다. KUBECONFIG 환경변수를 확인하거나 클러스터 내부에서 실행하세요.")
                return None
    except Exception as e:
         print(f"오류: 쿠버네티스 설정 로드 중 예외 발생: {e}")
         return None

    # 2. CoreV1Api 클라이언트 생성
    v1 = client.CoreV1Api()

    try:
        # 3. 레이블 셀렉터로 노드 목록 조회
        print(f"정보: 레이블 '{label_selector}'을(를) 가진 노드를 검색합니다...")
        nodes = v1.list_node(label_selector=label_selector)

        if not nodes.items:
            print(f"오류: 레이블 '{label_selector}'을(를) 가진 노드를 찾을 수 없습니다.")
            return None

        # 4. 찾은 노드 중 첫 번째 노드 선택
        node = nodes.items[0]
        node_name = node.metadata.name
        print(f"정보: 노드 '{node_name}'을(를) 찾았습니다.")

        node_ip = None
        # 5. 노드의 주소 목록에서 선호하는 IP 유형(InternalIP/ExternalIP) 검색
        for addr in node.status.addresses:
            if addr.type == preferred_ip_type:
                node_ip = addr.address
                print(f"정보: 선호하는 IP 유형 '{preferred_ip_type}' 주소 '{node_ip}'을(를) 찾았습니다.")
                break

        # 6. 선호하는 IP 유형을 찾지 못한 경우, 다른 유형 시도 (선택적)
        if not node_ip:
            fallback_ip_type = "ExternalIP" if preferred_ip_type == "InternalIP" else "InternalIP"
            print(f"정보: '{preferred_ip_type}' 유형을 찾지 못했습니다. '{fallback_ip_type}' 유형을 검색합니다...")
            for addr in node.status.addresses:
                 if addr.type == fallback_ip_type:
                     node_ip = addr.address
                     print(f"정보: 대체 IP 유형 '{fallback_ip_type}' 주소 '{node_ip}'을(를) 찾았습니다.")
                     break

        if not node_ip:
            print(f"오류: 노드 '{node_name}'에서 '{preferred_ip_type}' 또는 대체 IP 주소를 찾을 수 없습니다.")
            return None

        return node_ip

    except ApiException as e:
        print(f"오류: 쿠버네티스 API 호출 중 오류 발생: {e}")
        return None
    except Exception as e:
        print(f"오류: 예상치 못한 오류 발생: {e}")
        return None

In [17]:
target_label = "APP=MONITORING"
# 클러스터 내부 또는 VPN 등 내부망에서 접속 시 InternalIP 선호
ip_type_to_find = "InternalIP"
# 외부 인터넷에서 직접 접속해야 하고 ExternalIP가 있다면 ExternalIP 선호
# ip_type_to_find = "ExternalIP"

print(f"\n'{target_label}' 레이블 노드의 '{ip_type_to_find}' 주소를 찾습니다...")

found_ip = find_node_ip_by_label(label_selector=target_label, preferred_ip_type=ip_type_to_find)

if found_ip:
    loki_node_port = 30010
    loki_endpoint = f"{found_ip}:{loki_node_port}"
    print("\n--- 결과 ---")
    print(f"찾은 노드 IP 주소: {found_ip}")
    print(f"Loki 서비스 접속 주소 (NodePort): {loki_endpoint}")
    print("이 주소를 사용하여 Loki 서비스에 접속할 수 있습니다.")
else:
    print("\n--- 결과 ---")
    print("요청한 조건을 만족하는 노드의 IP 주소를 찾지 못했습니다.")


'APP=MONITORING' 레이블 노드의 'InternalIP' 주소를 찾습니다...
정보: Kubeconfig 파일을 로드했습니다.
정보: 레이블 'APP=MONITORING'을(를) 가진 노드를 검색합니다...
정보: 노드 'host-10-200-201-128'을(를) 찾았습니다.
정보: 선호하는 IP 유형 'InternalIP' 주소 '10.200.201.128'을(를) 찾았습니다.

--- 결과 ---
찾은 노드 IP 주소: 10.200.201.128
Loki 서비스 접속 주소 (NodePort): 10.200.201.128:30010
이 주소를 사용하여 Loki 서비스에 접속할 수 있습니다.
